In [1]:
!pip install textarena smithery anthropic httpx mcp --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 6.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: textarena
    Found existing installation: TextArena 0.5.8
    Uninstalling TextArena-0.5.8:
      Successfully uninstalled TextArena-0.5.8


In [10]:
import textarena as ta
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variables
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
e2b_api_key= os.getenv("E2B_API_KEY")

In [5]:
from textarena.core import Agent
import textarena as ta
import asyncio
from typing import Optional

STANDARD_GAME_PROMPT = "You are a competitive game player. Make sure you read the game instructions carefully, and always follow the required format."

class AsyncAnthropicAgent(Agent):
    """Agent class using the Anthropic Claude API to generate responses asynchronously."""
    def __init__(self, model_name: str, system_prompt: Optional[str] = STANDARD_GAME_PROMPT, max_tokens: int = 1000, temperature: float = 0.9, verbose: bool = False):
        """
        Initialize the Anthropic agent.

        Args:
            model_name (str): The name of the Claude model (e.g., "claude-3-5-sonnet-20241022").
            system_prompt (Optional[str]): The system prompt to use (default: STANDARD_GAME_PROMPT).
            max_tokens (int): The maximum number of tokens to generate.
            temperature (float): The temperature for randomness in response generation.
            verbose (bool): If True, additional debug info will be printed.
        """
        super().__init__()
        self.model_name = model_name
        self.system_prompt = system_prompt
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.verbose = verbose
        
        try:
            import anthropic
        except ImportError:
            raise ImportError(
                "Anthropic package is required for AsyncAnthropicAgent. "
                "Install it with: pip install anthropic"
            )
            
        self.client = anthropic.AsyncAnthropic()
    
    async def _make_request(self, observation: str) -> str:
        """Make a single API request to Anthropic and return the generated message."""
        response = await self.client.messages.create(
            model=self.model_name,
            max_tokens=self.max_tokens,
            temperature=self.temperature,
            system=self.system_prompt,
            messages=[
                {"role": "user", "content": [{"type": "text", "text": observation}]}
            ]
        )
        
        return response.content[0].text.strip()
    
    async def _retry_request(self, observation: str, retries: int = 3, delay: int = 5) -> str:
        """
        Attempt to make an API request with retries.

        Args:
            observation (str): The input to process.
            retries (int): The number of attempts to try.
            delay (int): Seconds to wait between attempts.

        Raises:
            Exception: The last exception caught if all retries fail.
        """
        last_exception = None
        for attempt in range(1, retries + 1):
            try:
                response = await self._make_request(observation)
                if self.verbose:
                    print(f"\nObservation: {observation}\nResponse: {response}")
                return response
            except Exception as e:
                last_exception = e
                print(f"Attempt {attempt} failed with error: {e}")
                if attempt < retries:
                    await asyncio.sleep(delay)
        raise last_exception
    
    async def __call__(self, observation: str) -> str:
        """
        Process the observation using the Anthropic API and return the generated response.
        
        Args:
            observation (str): The input string to process.
        
        Returns:
            str: The generated response.
        """
        if not isinstance(observation, str):
            raise ValueError(f"Observation must be a string. Received type: {type(observation)}")
        return await self._retry_request(observation)

In [6]:
import textarena as ta
import smithery
import mcp
import os
import json


class MCPAgent(AsyncAnthropicAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.url = smithery.create_smithery_url(
            "wss://server.smithery.ai/e2b/ws", {"e2bApiKey": os.environ["E2B_API_KEY"]}
        )

    async def _make_request(self, observation: str) -> str:
        """Make a single API request to Anthropic and return the generated message."""
        async with smithery.websocket_client(self.url) as streams:
            async with mcp.client.session.ClientSession(*streams) as session:

                try:
                    tools_result = await session.list_tools()
                    tools = tools_result.model_dump()["tools"]

                    tools = [
                        {"input_schema": tool.pop("inputSchema"), **tool}
                        for tool in tools
                        if "inputSchema" in tool
                    ]

                    print("Available tools:", tools)

                    final_response_text = ""
                    is_tool_call_pending = True
                    messages = [
                        {
                            "role": "user",
                            "content": [{"type": "text", "text": observation}],
                        }
                    ]

                    # Loop to handle multiple tool calls in a conversation
                    while is_tool_call_pending:
                        response = await self.client.messages.create(
                            model=self.model_name,
                            max_tokens=self.max_tokens,
                            temperature=self.temperature,
                            system=self.system_prompt,
                            messages=messages,
                            tools=tools,
                        )

                        print("Response:", response)

                        # Check if there's a tool_use in the response
                        is_tool_call_pending = False
                        for content_block in response.content:
                            if content_block.type == "tool_use":
                                is_tool_call_pending = True

                                tool_name = content_block.name
                                tool_input = content_block.input
                                tool_id = content_block.id

                                print(f"Tool called: {tool_name}")
                                print(f"Tool input: {json.dumps(tool_input, indent=2)}")

                                # Execute the tool using MCP session
                                try:
                                    tool_result = await session.call_tool(
                                        tool_name, tool_input
                                    )

                                    # Convert tool result to string format for Anthropic
                                    # The content must be a string, not an object
                                    tool_result_dict = tool_result.model_dump()
                                except Exception as e:
                                    if "MCP error" in str(e):
                                        tool_result_dict = {"error": str(e)}

                                result_str = json.dumps(tool_result_dict)
                                print(f"Tool result: {result_str}")

                                # Add tool call and result to messages
                                messages.append(
                                    {
                                        "role": "assistant",
                                        "content": [content_block.model_dump()],
                                    }
                                )

                                # Add tool response to messages - content must be a string
                                messages.append(
                                    {
                                        "role": "user",
                                        "content": [
                                            {
                                                "type": "tool_result",
                                                "tool_use_id": tool_id,
                                                "content": result_str,  # Now it's a string
                                            }
                                        ],
                                    }
                                )
                            elif content_block.type == "text":
                                # Accumulate text responses
                                final_response_text += content_block.text

                        # If no tool calls were made, we use the text response
                        if not is_tool_call_pending and not final_response_text:
                            final_response_text = response.content[0].text

                except Exception as e:

                    print(f"Error: {e}")
                    raise e

            return final_response_text.strip()

In [11]:
import textarena as ta

# Initialize agents
agents = {
    0: MCPAgent(model_name="claude-3-7-sonnet-20250219"),
    1: AsyncAnthropicAgent(model_name="claude-3-5-haiku-20241022"),
}

# Initialize environment from subset and wrap it
env = ta.make(env_id="SimpleNegotiation-v0")
env = ta.wrappers.LLMObservationWrapper(env=env)
env = ta.wrappers.SimpleRenderWrapper(
    env=env,
    player_names={0: "sonnet", 1: "haiku"},
)

env.reset(num_players=len(agents))
done = False
while not done:
    player_id, observation = env.get_observation()
    action = await agents[player_id](observation)
    done, info = env.step(action=action)
    print("step complete")
rewards = env.close()
print(rewards)

Available tools: [{'input_schema': {'type': 'object', 'properties': {'code': {'type': 'string'}}, 'required': ['code'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, 'name': 'run_code', 'description': 'Run python code in a secure sandbox by E2B. Using the Jupyter Notebook syntax.'}]
Response: Message(id='msg_0181eNcRMgAh1vvhZTeyfRj8', content=[TextBlock(citations=None, text="I'll help you play the Negotiation Game as Player 0. First, let me analyze the resources we have and their values to determine the best trading strategy.", type='text'), ToolUseBlock(id='toolu_01BAPA1YjdwUT8Lrn8JHbzUL', input={'code': '# Let\'s analyze our resources and their values\nresources = {\n    \'Wheat\': {\'qty\': 21, \'value\': 5},\n    \'Wood\': {\'qty\': 23, \'value\': 8},\n    \'Sheep\': {\'qty\': 6, \'value\': 18},\n    \'Brick\': {\'qty\': 21, \'value\': 24},\n    \'Ore\': {\'qty\': 23, \'value\': 33}\n}\n\n# Calculate total value\ntotal_value = sum(res[\'qty\'

╭────────────────────────────────────────────────── Game State ───────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭─────────────── Basic Information ───────────────╮ ╭─────────────────── player_resources ────────────────────╮ │
│ │                                                 │ │                                                         │ │
│ │                ╷                                │ │          ╷       ╷     ╷       ╷       ╷                │ │
│ │   Key          │ Value                          │ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood           │ │
│ │  ══════════════╪═══════                         │ │  ════════╪═══════╪═════╪═══════╪═══════╪══════          │ │
│ │   current_turn │ 1                              │ │   sonnet │ 21    │ 23  │ 6     │ 21    │ 23             │ │
│ │   max_turns    │ 10                             │ │   haiku  │ 17    │ 12  │ 6     │ 8     │ 19             │ │
│ │                ╵                                │ │          ╵       ╵     ╵       ╵       ╵                │ │
│ │                                                 │ │                                                         │ │
│ ╰─────────────────────────────────────────────────╯ ╰─────────────────────────────────────────────────────────╯ │
│ ╭──────────────── inventory_value ────────────────╮                                                             │
│ │                                                 │                                                             │
│ │          ╷        ╷         ╷                   │                                                             │
│ │   Player │ change │ current │ initial           │                                                             │
│ │  ════════╪════════╪═════════╪═════════          │                                                             │
│ │   sonnet │ 0      │ 1660    │ 1660              │                                                             │
│ │   haiku  │ 0      │ 1169    │ 1169              │                                                             │
│ │          ╵        ╵         ╵                   │                                                             │
│ │                                                 │                                                             │
│ ╰─────────────────────────────────────────────────╯                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
╭──────────────────────── Game State ────────────────────────╮╭──────────────────── Game Log ─────────────────────╮
│                                                            ││                                                   │
│ ╭────────────────── Basic Information ───────────────────╮ ││ [GAME]: Game started.                             │
│ │                                                        │ ││ sonnet: I'll help you play the Negotiation Game   │
│ │                ╷                                       │ ││ as Player 0. First, let me analyze the resources  │
│ │   Key          │ Value                                 │ ││ we have and their                                 │
│ │  ══════════════╪═══════                                │ ││ values to determine the best trading              │
│ │   current_turn │ 1                                     │ ││ strategy.Hello! I'm Player 0 in the Negotiation   │
│ │   max_turns    │ 10                                    │ ││ Game. Looking at my resources,                    │
│ │                ╵                                       │ ││ I have a good amount of Wheat, Wood, and Ore, but │
│ │                                                        │ ││ I could use more high-value resources to maximize │
│ ╰────────────────────────────────────────────────────────╯ ││ my total value.                                   │
│ ╭─────────────────── player_resources ───────────────────╮ ││                                                   │
│ │                                                        │ ││ I notice that Wheat and Wood are my lowest value  │
│ │          ╷       ╷     ╷       ╷       ╷               │ ││ resources (5 and 8 per unit), while Brick and Ore │
│ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood          │ ││ are my highest                                    │
│ │  ════════╪═══════╪═════╪═══════╪═══════╪══════         │ ││ value resources (24 and 33 per unit). It would be │
│ │   sonnet │ 21    │ 23  │ 6     │ 21    │ 23            │ ││ beneficial for me to trade some of my lower-value │
│ │   haiku  │ 17    │ 12  │ 6     │ 8     │ 19            │ ││ resources for                                     │
│ │          ╵       ╵     ╵       ╵       ╵               │ ││ higher-value ones.                                │
│ │                                                        │ ││                                                   │
│ ╰────────────────────────────────────────────────────────╯ ││ I'd like to propose a trade that would be         │
│ ╭─────────────────── inventory_value ────────────────────╮ ││ beneficial for both of us.                        │
│ │                                                        │ ││                                                   │
│ │          ╷        ╷         ╷                          │ ││ [Offer: 5 Wheat, 3 Wood -> 2 Brick]               │
│ │   Player │ change │ current │ initial                  │ ││                                                   │
│ │  ════════╪════════╪═════════╪═════════                 │ ││ This trade would give you 5 Wheat and 3 Wood in   │
│ │   sonnet │ 0      │ 1660    │ 1660                     │ ││ exchange for 2 Brick. I have plenty of Wheat and  │
│ │   haiku  │ 0      │ 1169    │ 1169                     │ ││ Wood to spare, and                                │
│ │          ╵        ╵         ╵                          │ ││ Brick has a higher value for me. Let me know if   │
│ │                                                        │ ││ this works for you or if you'd prefer a different │
│ ╰────────────────────────────────────────────────────────╯ ││ trade!                                            │
│                                                            ││ [GAME]: Player 0 made the following offer to      │
│                                                       

step complete


╭────────────────────────────────────────────────── Game State ───────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭─────────────── Basic Information ───────────────╮ ╭─────────────────── player_resources ────────────────────╮ │
│ │                                                 │ │                                                         │ │
│ │                ╷                                │ │          ╷       ╷     ╷       ╷       ╷                │ │
│ │   Key          │ Value                          │ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood           │ │
│ │  ══════════════╪═══════                         │ │  ════════╪═══════╪═════╪═══════╪═══════╪══════          │ │
│ │   current_turn │ 2                              │ │   sonnet │ 21    │ 23  │ 6     │ 21    │ 23             │ │
│ │   max_turns    │ 10                             │ │   haiku  │ 17    │ 12  │ 6     │ 8     │ 19             │ │
│ │                ╵                                │ │          ╵       ╵     ╵       ╵       ╵                │ │
│ │                                                 │ │                                                         │ │
│ ╰─────────────────────────────────────────────────╯ ╰─────────────────────────────────────────────────────────╯ │
│ ╭──────────────── inventory_value ────────────────╮                                                             │
│ │                                                 │                                                             │
│ │          ╷        ╷         ╷                   │                                                             │
│ │   Player │ change │ current │ initial           │                                                             │
│ │  ════════╪════════╪═════════╪═════════          │                                                             │
│ │   sonnet │ 0      │ 1660    │ 1660              │                                                             │
│ │   haiku  │ 0      │ 1169    │ 1169              │                                                             │
│ │          ╵        ╵         ╵                   │                                                             │
│ │                                                 │                                                             │
│ ╰─────────────────────────────────────────────────╯                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
╭──────────────────────── Game State ────────────────────────╮╭──────────────────── Game Log ─────────────────────╮
│                                                            ││                                                   │
│ ╭────────────────── Basic Information ───────────────────╮ ││ don't have many of.                               │
│ │                                                        │ ││                                                   │
│ │                ╷                                       │ ││ [Accept]                                          │
│ │   Key          │ Value                                 │ ││                                                   │
│ │  ══════════════╪═══════                                │ ││ Thank you for the offer. I'll accept this trade   │
│ │   current_turn │ 9                                     │ ││ of 3 Brick for my 1 Ore and 2 Sheep. While it's   │
│ │   max_turns    │ 10                                    │ ││ only a small value                                │
│ │                ╵                                       │ ││ increase for me, I'm building up my Brick         │
│ │                                                        │ ││ reserves which gives me flexibility for future    │
│ ╰────────────────────────────────────────────────────────╯ ││ trades. Looking forward to                        │
│ ╭─────────────────── player_resources ───────────────────╮ ││ continuing our trading relationship!              │
│ │                                                        │ ││ [GAME]: Player 0 accepted the trade offer from    │
│ │          ╷       ╷     ╷       ╷       ╷               │ ││ Player 1.                                         │
│ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood          │ ││ haiku: I understand I'm Player 1 in the           │
│ │  ════════╪═══════╪═════╪═══════╪═══════╪══════         │ ││ Negotiation Game. I'll continue my strategic      │
│ │   sonnet │ 26    │ 21  │ 5     │ 19    │ 24            │ ││ approach to trading.                              │
│ │   haiku  │ 12    │ 14  │ 7     │ 10    │ 18            │ ││                                                   │
│ │          ╵       ╵     ╵       ╵       ╵               │ ││ After the previous trade, my updated resources    │
│ │                                                        │ ││ are:                                              │
│ ╰────────────────────────────────────────────────────────╯ ││         + [Wheat]   Qty: 8    Value: 6            │
│ ╭─────────────────── inventory_value ────────────────────╮ ││         + [Wood]    Qty: 19   Value: 11           │
│ │                                                        │ ││         + [Sheep]   Qty: 9    Value: 14           │
│ │          ╷        ╷         ╷                          │ ││         + [Brick]   Qty: 12   Value: 24           │
│ │   Player │ change │ current │ initial                  │ ││         + [Ore]     Qty: 12   Value: 35           │
│ │  ════════╪════════╪═════════╪═════════                 │ ││                                                   │
│ │   sonnet │ 34     │ 1694    │ 1660                     │ ││ I notice I have a good quantity of Wood, which is │
│ │   haiku  │ -35    │ 1134    │ 1169                     │ ││ one of my lower-value resources. I'll look to     │
│ │          ╵        ╵         ╵                          │ ││ trade some Wood for                               │
│ │                                                        │ ││ more valuable resources.                          │
│ ╰────────────────────────────────────────────────────────╯ ││                                                   │
│                                                            ││ [Offer: 5 Wood -> 1 Ore, 1 Sheep]                 │
│                                                       

step complete


╭────────────────────────────────────────────────── Game State ───────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭─────────────── Basic Information ───────────────╮ ╭─────────────────── player_resources ────────────────────╮ │
│ │                                                 │ │                                                         │ │
│ │                ╷                                │ │          ╷       ╷     ╷       ╷       ╷                │ │
│ │   Key          │ Value                          │ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood           │ │
│ │  ══════════════╪═══════                         │ │  ════════╪═══════╪═════╪═══════╪═══════╪══════          │ │
│ │   current_turn │ 9                              │ │   sonnet │ 26    │ 21  │ 5     │ 19    │ 24             │ │
│ │   max_turns    │ 10                             │ │   haiku  │ 12    │ 14  │ 7     │ 10    │ 18             │ │
│ │                ╵                                │ │          ╵       ╵     ╵       ╵       ╵                │ │
│ │                                                 │ │                                                         │ │
│ ╰─────────────────────────────────────────────────╯ ╰─────────────────────────────────────────────────────────╯ │
│ ╭──────────────── inventory_value ────────────────╮                                                             │
│ │                                                 │                                                             │
│ │          ╷        ╷         ╷                   │                                                             │
│ │   Player │ change │ current │ initial           │                                                             │
│ │  ════════╪════════╪═════════╪═════════          │                                                             │
│ │   sonnet │ 34     │ 1694    │ 1660              │                                                             │
│ │   haiku  │ -35    │ 1134    │ 1169              │                                                             │
│ │          ╵        ╵         ╵                   │                                                             │
│ │                                                 │                                                             │
│ ╰─────────────────────────────────────────────────╯                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
╭──────────────────────── Game State ────────────────────────╮╭──────────────────── Game Log ─────────────────────╮
│                                                            ││                                                   │
│ ╭────────────────── Basic Information ───────────────────╮ ││                                                   │
│ │                                                        │ ││ I'm proposing to trade 5 Wood (valued at 55) for  │
│ │                ╷                                       │ ││ 1 Ore (valued at 35) and 1 Sheep (valued at 14),  │
│ │   Key          │ Value                                 │ ││ which totals 49.                                  │
│ │  ══════════════╪═══════                                │ ││ This trade allows me to convert my lower-value    │
│ │   current_turn │ 9                                     │ ││ Wood into more strategically valuable resources.  │
│ │   max_turns    │ 10                                    │ ││ The trade slightly                                │
│ │                ╵                                       │ ││ favors me in terms of resource value, providing   │
│ │                                                        │ ││ an incentive for Player 0 to consider the         │
│ ╰────────────────────────────────────────────────────────╯ ││ exchange.                                         │
│ ╭─────────────────── player_resources ───────────────────╮ ││                                                   │
│ │                                                        │ ││ Waiting for Player 0's response.                  │
│ │          ╷       ╷     ╷       ╷       ╷               │ ││ [GAME]: Player 1 made the following offer to      │
│ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood          │ ││ Player 0: Offered items: 5 Wood -> Requested      │
│ │  ════════╪═══════╪═════╪═══════╪═══════╪══════         │ ││ items: 1 Ore, 1 Sheep                             │
│ │   sonnet │ 26    │ 21  │ 5     │ 19    │ 24            │ ││ sonnet: I'll analyze this trade offer to          │
│ │   haiku  │ 12    │ 14  │ 7     │ 10    │ 18            │ ││ determine if it's beneficial for me as Player     │
│ │          ╵       ╵     ╵       ╵       ╵               │ ││ 0.I'll analyze this trade                         │
│ │                                                        │ ││ offer and make a strategic decision.              │
│ ╰────────────────────────────────────────────────────────╯ ││                                                   │
│ ╭─────────────────── inventory_value ────────────────────╮ ││ I've carefully evaluated the trade from Player 1: │
│ │                                                        │ ││                                                   │
│ │          ╷        ╷         ╷                          │ ││ Looking at the numbers:                           │
│ │   Player │ change │ current │ initial                  │ ││ - I would give: 1 Ore (worth 33) and 1 Sheep      │
│ │  ════════╪════════╪═════════╪═════════                 │ ││ (worth 18) = 51 total value                       │
│ │   sonnet │ 34     │ 1694    │ 1660                     │ ││ - I would receive: 5 Wood (worth 40) = 40 total   │
│ │   haiku  │ -35    │ 1134    │ 1169                     │ ││ value                                             │
│ │          ╵        ╵         ╵                          │ ││ - This gives me a net loss of 11 value points     │
│ │                                                        │ ││                                                   │
│ ╰────────────────────────────────────────────────────────╯ ││ After my previous trades, I've built up a good    │
│                                                            ││ amount of Brick (26 units worth 624), but I'm     │
│                                                       

step complete
{0: 1, 1: -1}
